In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)

filter_param = catalog.load('params:openaire_fetch_options.filter_param')
filter_value = catalog.load('params:openaire_fetch_options.filter_value')
df = catalog.load('raw/openaire/researchproduct#parquet')

[02/28/25 10:01:15] INFO     Loading data from params:openaire_fetch_options.filter_param       ]8;id=558993;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=350558;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_fetch_options.filter_value       ]8;id=309098;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=302601;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from raw/openaire/researchproduct#parquet             ]8;id=826679;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=115894;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [2]:
df.columns


Index(['author', 'openAccessColor', 'publiclyFunded', 'type', 'language',
       'subjects', 'mainTitle', 'description', 'publicationDate', 'publisher',
       'source', 'format', 'bestAccessRight', 'container', 'id', 'originalId',
       'indicators', 'instance', 'isGreen', 'isInDiamondJournal',
       'contributor', 'pid', 'country', 'contactPerson', 'embargoEndDate',
       'coverage', 'version', 'size', 'filter_applied'],
      dtype='object')

## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [ ]:
def land_openaire_map_researchproduct_author(df: pd.DataFrame)-> pd.DataFrame:

    expected_columns = [
        'id',
        'author',
    ]

    # Agregar columnas faltantes con NaN
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[expected_columns].copy()
    df.reset_index(drop=True, inplace=True)

    ## author
    df_researchproduct2author = df.explode('author').reset_index(drop=True)
    df_researchproduct2author = df_researchproduct2author[['id','author']]
    df_authors = pd.json_normalize(df_researchproduct2author['author']).reset_index(drop=True)
    df_researchproduct2author = pd.concat([df_researchproduct2author.drop(columns=['author']), df_authors], axis=1)

    df_researchproduct2author['load_datetime'] = date.today()

    return df_researchproduct2author


In [ ]:
df_researchproduct2author = land_openaire_map_researchproduct_author(filter_param, filter_value, df)

In [5]:
df_researchproduct2author

,id,fullName,name,pid,rank,surname,pid.id.scheme,pid.id.value,pid.provenance,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Steffen, Kevin Denis",Kevin Denis,NaN,1.0,Steffen,NaN,NaN,NaN,2025-02-28
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Gortari Castillo, Lihuel",Lihuel,NaN,2.0,Gortari Castillo,NaN,NaN,NaN,2025-02-28
2,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Cordiviola, Carlos Ángel",Carlos Ángel,NaN,3.0,Cordiviola,NaN,NaN,NaN,2025-02-28
3,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Moré, Gastón",Gastón,NaN,4.0,Moré,NaN,NaN,NaN,2025-02-28
4,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Arias, Ruben Omar",Ruben Omar,NaN,5.0,Arias,NaN,NaN,NaN,2025-02-28
...,...,...,...,...,...,...,...,...,...,...
844062,portalunbbra::fe22079ac12e4b48e955751af4ce027b,"GENTILE, Ana María",Ana María,NaN,1.0,Gentile,NaN,NaN,NaN,2025-02-28
844063,revistasumft::106b3b4f6115356de22e7aaf07960b34,"Canzani, Ana Inés",Ana Inés,NaN,1.0,Canzani,NaN,NaN,NaN,2025-02-28
844064,sharePrprorg::b68010ac09fbe1824d423110978b672f,"Fernández-Bedmar, Zahira",Zahira,NaN,1.0,Fernández-Bedmar,NaN,NaN,NaN,2025-02-28
844065,sharePrprorg::b68010ac09fbe1824d423110978b672f,"Demyda-Peyrás, Sebastián",Sebastián,NaN,2.0,Demyda-Peyrás,NaN,NaN,NaN,2025-02-28
